# Table of Contents
 <p><div class="lev1 toc-item"><a href="#setup" data-toc-modified-id="setup-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>setup</a></div><div class="lev1 toc-item"><a href="#isIsomorphic-code" data-toc-modified-id="isIsomorphic-code-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>isIsomorphic code</a></div><div class="lev1 toc-item"><a href="#compare-time-of-original-algorithm" data-toc-modified-id="compare-time-of-original-algorithm-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>compare time of original algorithm</a></div><div class="lev1 toc-item"><a href="#compare-time-of-new-algorithm" data-toc-modified-id="compare-time-of-new-algorithm-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>compare time of new algorithm</a></div>

This notebook compares the efficiency of two methods to compare reaction isomorphism, to see the time benefits of it. Two similar H_abstraction reactions are created and compared to look at time efficiency.

# setup

In [1]:
from rmgpy.species import Species
from rmgpy.reaction import Reaction
from cProfile import run

In [2]:
r1 = Species().fromSMILES('C=CC=CC=CC=CC=C[CH2]')
r2 = Species().fromSMILES('C=CC=CC=CC=C[CH2]')
p1 = Species().fromSMILES('C=CC=CC=CC=CC')
p2a = Species().fromSMILES('C=CC=CC=CC=CC=C=C')
p2b = Species().fromSMILES('C=C=CC=CC=CC=C=CC')
species = [r1,r2,p1,p2a,p2b]
for s in species: s.generateResonanceIsomers()

In [39]:
rxn1 = Reaction(reactants=[r1,r2], products = [p1, p2a])
rxn2 = Reaction(reactants=[r1,r2], products = [p1, p2b])
rxn1copy = rxn1.copy()
rxn2swapped_reactants = Reaction(reactants=[r2,r1], products = [p2b, p1])


# isIsomorphic code

In [40]:
def isIsomorphic(rxn1, other, eitherDirection=True, checkIdentical = False):
        """
        Return ``True`` if this reaction is the same as the `other` reaction,
        or ``False`` if they are different. The comparison involves comparing
        isomorphism of reactants and products, and doesn't use any kinetic
        information.

        If `eitherDirection=False` then the directions must match.
        """

        # Compare reactants to reactants
        forwardReactantsMatch = _isomorphicSpeciesList(rxn1.reactants, 
                                    other.reactants,checkIdentical = checkIdentical)
        
        # Compare products to products
        forwardProductsMatch = _isomorphicSpeciesList(rxn1.products, 
                                    other.products,checkIdentical = checkIdentical)

        # Return now, if we can
        if (forwardReactantsMatch and forwardProductsMatch):
            return True
        if not eitherDirection:
            return False
        
        # Compare reactants to products
        reverseReactantsMatch = _isomorphicSpeciesList(rxn1.reactants, 
                                    other.products,checkIdentical = checkIdentical)

        # Compare products to reactants
        reverseProductsMatch = _isomorphicSpeciesList(rxn1.products, 
                                    other.reactants,checkIdentical = checkIdentical)

        # should have already returned if it matches forwards, or we're not allowed to match backwards
        return  (reverseReactantsMatch and reverseProductsMatch)

# compare time of original algorithm

In [41]:
def _isomorphicSpeciesList(list1, list2, checkIdentical=False):
    """
    This method compares whether lists of species or molecules are isomorphic
    or identical. It is used for the 'isIsomorphic' method of Reaction class.
    It likely can be useful elswehere as well:
        
        list1 - list of species/molecule objects of reaction1
        list2 - list of species/molecule objects of reaction2
        checkIdentical - if true, uses the 'isIdentical' comparison
                         if false, uses the 'isIsomorphic' comparison
                         
    Returns True if the lists are isomorphic/identical & false otherwise
    """

    def comparison_method(other1, other2, checkIdentical=checkIdentical):
        if checkIdentical:
            return other1.isIdentical(other2)
        else:
            return other1.isIsomorphic(other2)

    if len(list1) == len(list2) == 1:
        if comparison_method(list1[0], list2[0]):
            return True
    elif len(list1) == len(list2) == 2:
        if comparison_method(list1[0], list2[0]) \
                    and comparison_method(list1[1], list2[1]):
            return True
        elif comparison_method(list1[0], list2[1]) \
                    and comparison_method(list1[1], list2[0]):
            return True
    elif len(list1) == len(list2) == 3:
        if (    comparison_method(list1[0], list2[0]) and
                comparison_method(list1[1], list2[1]) and
                comparison_method(list1[2], list2[2]) ):
            return True
        elif (  comparison_method(list1[0], list2[0]) and
                comparison_method(list1[1], list2[2]) and
                comparison_method(list1[2], list2[1]) ):
            return True
        elif (  comparison_method(list1[0], list2[1]) and
                comparison_method(list1[1], list2[0]) and
                comparison_method(list1[2], list2[2]) ):
            return True
        elif (  comparison_method(list1[0], list2[2]) and
                comparison_method(list1[1], list2[0]) and
                comparison_method(list1[2], list2[1]) ):
            return True
        elif (  comparison_method(list1[0], list2[1]) and
                comparison_method(list1[1], list2[2]) and
                comparison_method(list1[2], list2[0]) ):
            return True
        elif (  comparison_method(list1[0], list2[2]) and
                comparison_method(list1[1], list2[1]) and
                comparison_method(list1[2], list2[0]) ):
            return True
    elif len(list1) == len(list2):
        raise NotImplementedError("Can't check isomorphism of lists with {0} species/molecules".format(len(list1)))
    # nothing found
    return False

In [42]:
run('for i in range(10000): isIsomorphic(rxn1,rxn2)',)

         300003 function calls in 0.799 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    10000    0.017    0.000    0.795    0.000 <ipython-input-40-95d0327e9f94>:1(isIsomorphic)
    40000    0.059    0.000    0.778    0.000 <ipython-input-41-d3da26f90ae7>:1(_isomorphicSpeciesList)
    90000    0.710    0.000    0.710    0.000 <ipython-input-41-d3da26f90ae7>:15(comparison_method)
        1    0.004    0.004    0.799    0.799 <string>:1(<module>)
   160000    0.008    0.000    0.008    0.000 {len}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
        1    0.000    0.000    0.000    0.000 {range}




In [43]:
run('for i in range(10000): isIsomorphic(rxn1,rxn2swapped_reactants)')

         310003 function calls in 0.877 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    10000    0.018    0.000    0.872    0.000 <ipython-input-40-95d0327e9f94>:1(isIsomorphic)
    40000    0.062    0.000    0.855    0.000 <ipython-input-41-d3da26f90ae7>:1(_isomorphicSpeciesList)
   100000    0.784    0.000    0.784    0.000 <ipython-input-41-d3da26f90ae7>:15(comparison_method)
        1    0.005    0.005    0.877    0.877 <string>:1(<module>)
   160000    0.008    0.000    0.008    0.000 {len}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
        1    0.000    0.000    0.000    0.000 {range}




In [44]:
run('for i in range(10000): isIsomorphic(rxn1,rxn1copy)')

         150003 function calls in 0.970 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    10000    0.011    0.000    0.965    0.000 <ipython-input-40-95d0327e9f94>:1(isIsomorphic)
    20000    0.031    0.000    0.954    0.000 <ipython-input-41-d3da26f90ae7>:1(_isomorphicSpeciesList)
    40000    0.919    0.000    0.919    0.000 <ipython-input-41-d3da26f90ae7>:15(comparison_method)
        1    0.005    0.005    0.970    0.970 <string>:1(<module>)
    80000    0.004    0.000    0.004    0.000 {len}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
        1    0.000    0.000    0.000    0.000 {range}




# compare time of new algorithm

In [45]:
def _isomorphicSpeciesList(list1, list2, checkIdentical=False):
    """
    This method compares whether lists of species or molecules are isomorphic
    or identical. It is used for the 'isIsomorphic' method of Reaction class.
    It likely can be useful elswehere as well:
        
        list1 - list of species/molecule objects of reaction1
        list2 - list of species/molecule objects of reaction2
        checkIdentical - if true, uses the 'isIdentical' comparison
                         if false, uses the 'isIsomorphic' comparison
                         
    Returns True if the lists are isomorphic/identical & false otherwise
    """

    def comparison_method(other1, other2, checkIdentical=checkIdentical):
        if checkIdentical:
            return other1.isIdentical(other2)
        else:
            return other1.isIsomorphic(other2)

    list2Temp = list2[:]
    for entry in list1:
       for index, entry2 in enumerate(list2Temp):
           if comparison_method(entry,entry2):
               del list2Temp[index]
               break
       else: #loop doesn't break, something is missing
             return False
    #all loops completed
    return True

In [46]:
run('for i in range(10000): isIsomorphic(rxn1,rxn2)',)

         130003 function calls in 0.817 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    10000    0.019    0.000    0.812    0.000 <ipython-input-40-95d0327e9f94>:1(isIsomorphic)
    40000    0.067    0.000    0.793    0.000 <ipython-input-45-2de5dad1ee6b>:1(_isomorphicSpeciesList)
    80000    0.726    0.000    0.726    0.000 <ipython-input-45-2de5dad1ee6b>:15(comparison_method)
        1    0.005    0.005    0.817    0.817 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
        1    0.000    0.000    0.000    0.000 {range}




In [47]:
run('for i in range(10000): isIsomorphic(rxn1,rxn2swapped_reactants)')

         150003 function calls in 0.887 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    10000    0.019    0.000    0.882    0.000 <ipython-input-40-95d0327e9f94>:1(isIsomorphic)
    40000    0.071    0.000    0.863    0.000 <ipython-input-45-2de5dad1ee6b>:1(_isomorphicSpeciesList)
   100000    0.792    0.000    0.792    0.000 <ipython-input-45-2de5dad1ee6b>:15(comparison_method)
        1    0.005    0.005    0.887    0.887 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
        1    0.000    0.000    0.000    0.000 {range}




In [48]:
run('for i in range(10000): isIsomorphic(rxn1,rxn1copy)')

         70003 function calls in 1.005 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    10000    0.012    0.000    0.999    0.000 <ipython-input-40-95d0327e9f94>:1(isIsomorphic)
    20000    0.048    0.000    0.988    0.000 <ipython-input-45-2de5dad1ee6b>:1(_isomorphicSpeciesList)
    40000    0.940    0.000    0.940    0.000 <ipython-input-45-2de5dad1ee6b>:15(comparison_method)
        1    0.005    0.005    1.005    1.005 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
        1    0.000    0.000    0.000    0.000 {range}


